# Python et C Sharp

<!--RST ..index:: C# !RST-->

Je n'ai pas réussi à trouver un moyen d'utiliser [C#](http://fr.wikipedia.org/wiki/C_sharp) directement depuis un notebook même si cela est possible avec [F#](http://fr.wikipedia.org/wiki/F_Sharp) : [BayardRock/IfSharp](https://github.com/BayardRock/IfSharp) (voir la démo [IPython notebook backed by an F# kernel](http://nbviewer.ipython.org/github/BayardRock/IfSharp/blob/master/Feature%20Notebook.ipynb)).

A défaut, ce notebook démontre une façon d'appeler les fonctions d'une DLL écrite en C#. Cela se fait grâce au module [pythonnet](https://github.com/renshawbay/pythonnet). Le concepteur ne maintient plus la librairie mais d'autres le font ([Christoph Gohlke's page](http://www.lfd.uci.edu/~gohlke/pythonlibs/#pythonnet)). Elle est aussi accessible dans le module [ensae_teaching_cs](https://github.com/sdpython/ensae_teaching_cs).

Voici quelques exemples ce qu'on peut faire :

In [1]:
from ensae_teaching_cs.pythonnet import clr

In [2]:
from System import Double
d = Double(5.5)
d

In [3]:
from System.Collections.Generic import List
a = List[Double]()
a

<System.Collections.Generic.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]] at 0x8ba0c18>

In [4]:
a.Add(10)
a

<System.Collections.Generic.0, Culture=neutral, PublicKeyToken=b77a5c561934e089]] at 0x8ba0c18>

On ne peut pas écrire du C#, et donc pas de [Linq](http://msdn.microsoft.com/fr-fr/library/bb397926.aspx) non plus :

In [5]:
from System import String
String.Join(",", a.Select(c=>c.ToString()).ToArray())

SyntaxError: invalid syntax (<ipython-input-5-7fa1e78b6628>, line 2)

Ce module est surtout intéressant pour appeler depuis Python des fonctionnalités codées avec [.net](http://fr.wikipedia.org/wiki/Microsoft_.NET). Il suffit d'utiliser la fonction [AddReference](http://stackoverflow.com/questions/14520888/clr-addreferenceexample-file-unable-to-find-assembly). Le chemin de la DLL doit être inclus dans ``sys.path`` :

In [7]:
import os
from clr import AddReference
from ensae_teaching_cs.pythonnet import csdll
import sys
sys.path.append(os.path.dirname(csdll.__file__))
AddReference("ENSAE.Voice")
from ENSAE.Voice import Speech
Speech.VocalSynthesis("ENSAE", "fr-FR","","")

Jupyter permet de définir [magic command](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/jupyter_custom_magics.html). Quelques-unes sont implémentées dans le module [ensae_teaching_cs.mypython.custom_magic](http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/ensae_teaching_cs/mypython/custom_magics.html#module-mypython.custom_magics). 

In [8]:
from ensae_teaching_cs.mypython.custom_magics import register_magics
register_magics()

On peut comme ceci cacher une partie de la logique C# qui permet d'appeler la synthèse vocale :

In [9]:
%%SPEAK fr-FR
Ceci est une commande magique IPython.
La synthèse vocale est activée dès que la cellule est exécutée.

On peut définir un autre type de cellule magique qui compile du code C# de telle sorte que celui-ci apparaissent comme une fonction Python :

In [10]:
%%CS puissance System.dll
public static double puissance(double x, double y)
{
    if (y == 0) return 1.0 ;
    return System.Math.Pow(x,y) ;
}

<function ensae_teaching_cs.td_2a.pythoncs.create_cs_function.<locals>.<lambda>>

In [11]:
puissance(3.0,3.0)

27.0

L'utilisation du C# est parfois délicate car C# est rigoureux sur les types. Il n'y aura pas de conversion implicite lors de l'application à la fonction.

In [13]:
puissance(3.0,3)

0.037037037037037035

Vous trouverez quelques petits détails à connaître ici [n issue or something to know with pythonnet](http://www.xavierdupre.fr/blog/xd_blog_nojs.html) et là [Using pythonnet](http://www.xavierdupre.fr/blog/2014-10-23_nojs.html). La création de la commande magique ``%%CS`` est détaillée dans cet article [Magic command %%CS](http://www.xavierdupre.fr/blog/2014-09-20_nojs.html).

**Remarque : C# est fortement typé**

Les listes de Python peuvent contenir des valeurs de type différent. Ce n'est pas le cas en C# et cela pose souvent des problèmes lors de l'appel d'une fonction C#. Parfois, le plus simple est de [sérialiser](http://fr.wikipedia.org/wiki/S%C3%A9rialisation) les données sous forme de chaînes de caractères.